In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Essential Imports

In [ ]:
import tensorflow as tf
from keras.regularizers import l2
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## Essential Constants

In [ ]:
train_path = '/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/train/'
test_path = '/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/test/'

# useful for getting number of classes
folders = glob('/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/train/*')

num_classes=len(folders)

# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [ ]:
print(num_classes)

2


## Creating a new model

In [ ]:
model = Sequential()
model.add(VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(1, kernel_regularizer=l2(0.01),activation='linear')) # Final Layer is SVM Layer
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer='adam', loss='hinge', metrics=['accuracy',tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.Hinge(),tf.keras.metrics.BinaryCrossentropy()])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3211392   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 17,926,209
Trainable params: 3,211,521
Non-trainable params: 14,714,688
_________________________________________________________________


## Setting up training and testing data

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_set = data_generator.flow_from_directory(train_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')
test_set = data_generator.flow_from_directory(test_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')

Found 3000 images belonging to 2 classes.
Found 1400 images belonging to 2 classes.


In [ ]:
print(test_set)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
# checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=20, verbose=1, mode='auto')

In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
#   callbacks=[early]
)

f=open('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-SI-VGG16-stats.txt','w')
f.write(str(r.history))
f.close()

model.save('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-SI-VGG16-model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
300/300 [==============================] - 1864s 6s/step - loss: 2.8738 - accuracy: 0.6540 - mean_absolute_error: 6.3902 - mean_squared_error: 156.9382 - hinge: 2.8594 - binary_crossentropy: 4.6697 - val_loss: 0.5528 - val_accuracy: 0.8457 - val_mean_absolute_error: 3.4426 - val_mean_squared_error: 25.8370 - val_hinge: 0.5422 - val_binary_crossentropy: 1.9260
Epoch 2/20
300/300 [==============================] - 34s 114ms/step - loss: 0.6307 - accuracy: 0.8109 - mean_absolute_error: 4.0895 - mean_squared_error: 32.5316 - hinge: 0.6206 - binary_crossentropy: 2.5450 - val_loss: 0.3041 - val_accuracy: 0.8757 - val_mean_absolute_error: 3.3750 - val_mean_squared_error: 25.9272 - val_hinge: 0.2957 - val_binary_crossentropy: 1.4140
Epoch 3/20
300/300 [==============================] - 33s 111ms/step - loss: 0.7868 - accuracy: 0.7811 - mean_absolute_error: 4.5797 - mean_squared_error: 35.5314 - hinge: 0.7787 - binary_crossentropy: 3.0121 - val_loss: 0.3470 - val_accuracy: 0.8529 - v